In [1]:
import pandas as pd
import numpy as np

# get_data

In [2]:
def get_data(path, file_name):
    return pd.read_csv(f'../{path}/{file_name}.csv')

# extract_sample

In [7]:
def extract_sample(X):
    grouped = X.groupby('mmsi')

    # Get the first group's key (value in the 'mmsi' column)
    first_group_key = list(grouped.groups.keys())[0]

    # Get the first group as a DataFrame
    first_group_df = pd.DataFrame(grouped.get_group(first_group_key))
    
    first_group_df_cleaned = first_group_df.drop(columns=['source', 'is_fishing'])
    
    return first_group_df_cleaned
        


# Send CSV sample

In [4]:
def send_csv(df, file_name):
    output_folder = '../mlops/data/sample_data'
    output_file = f'sample_{file_name}.csv'

    # Construct the full path
    output_path = f'{output_folder}/{output_file}'

    # Save the DataFrame to the specified path
    df.to_csv(output_path, index=False)

# Extract all sample to .csv

In [5]:
path='data/raw_data'
file_names = ['trawlers', 'drifting_longlines', 'fixed_gear', 'pole_and_line', 'purse_seines', 'trollers', 'unknown']

for file_name in file_names: 
    df = get_data(path, file_name)
    
    data = extract_sample(df)
    
    send_csv(data, file_name)

# test

In [23]:
path='data/raw_data'
file_name = 'fixed_gear'

df = get_data(path, file_name)
df = extract_sample(df)
df.shape

(65423, 8)

In [22]:
df.head()

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon
1118966,5.601266e+12,1.338394e+09,0.0,1999.950928,0.0,0.000000,-34.900513,-56.203678
1118967,5.601266e+12,1.338414e+09,0.0,1999.950928,0.0,0.000000,-34.900528,-56.203712
1118968,5.601266e+12,1.338430e+09,0.0,1999.950928,0.0,119.000000,-34.900593,-56.203705
1118969,5.601266e+12,1.338436e+09,0.0,1999.950928,0.0,308.399994,-34.900574,-56.203705
1118970,5.601266e+12,1.338459e+09,0.0,1999.950928,0.0,0.000000,-34.900509,-56.203693


In [24]:
df_fishing = df

# ## Date time (hour -> Angular distance)
df_fishing['timestamp'] = pd.to_datetime(df_fishing['timestamp'], unit='s')
df_fishing.rename(columns={"timestamp": "date"}, inplace=True)
df_fishing['hour'] = df_fishing['date'].dt.hour
df_fishing['month'] = df_fishing['date'].dt.month
df_fishing['day_of_week'] = df_fishing['date'].dt.day_of_week

#
df_fishing['hour_sin'] = np.sin(df_fishing['hour'] * (2 * np.pi / 24))
df_fishing['hour_cos'] = np.cos(df_fishing['hour'] * (2 * np.pi / 24))

# Dropping rows with NAN values
df_fishing_clean = df_fishing.dropna()

df_fishing_clean.shape


(65423, 13)

In [29]:
df_fishing.columns


X = df_fishing.drop(columns=['date','hour'])

X.columns

Index(['mmsi', 'distance_from_shore', 'distance_from_port', 'speed', 'course',
       'lat', 'lon', 'month', 'day_of_week', 'hour_sin', 'hour_cos'],
      dtype='object')

In [ ]:
Index(['mmsi', 'distance_from_shore', 'distance_from_port', 'speed', 'course',
       'lat', 'lon', 'month', 'day_of_week', 'hour_sin', 'hour_cos'],

In [25]:
number_vessel=df_fishing_clean['mmsi'].value_counts().count()
print('Total number of vessels in our small dataset: ', number_vessel)

Total number of vessels in our small dataset:  1


In [32]:
df_fishing_clean.groupby('mmsi').size()

mmsi
1.848346e+13    55187
4.733417e+13     7886
8.703142e+13    66986
1.020291e+14    13344
1.961844e+14     3896
2.145727e+14    14016
dtype: int64